In [1]:
import tensorflow as tf
import gym
import datetime
from tqdm import tqdm
import random
from go_ai import data, metrics, mcts, models, policies
import matplotlib.pyplot as plt

In [2]:
from absl import logging
logging._warn_preinit_stderr = 0
logging.set_verbosity(logging.INFO)

# Hyperparameters

In [3]:
BOARD_SIZE = 7

In [4]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 32
MAX_STEPS = 2 * BOARD_SIZE**2
BATCH_SIZE = 32

In [5]:
NUM_EVAL_GAMES = 32
ITERATIONS_PER_EVAL = 6

In [6]:
LEARNING_RATE = 1e-3

In [7]:
MC_SIMS = 0
TEMP_FUNC = lambda step: 1/(step+1)

In [8]:
WEIGHTS_DIR = 'model_weights/'
ACTOR_CRITIC_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
LOAD_SAVED_MODELS = True

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [9]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [10]:
!rm -rf ./logs/

Metrics

In [11]:
tb_metrics = {}
for metric_key in ['val_loss', 'overall_loss', 'num_steps', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/actor_critic/{}/main'.format(current_time)
summary_writer = tf.summary.create_file_writer(log_dir)    

# Machine Learning Models

In [13]:
actor_critic = models.make_actor_critic(BOARD_SIZE, 'val_net', 'tanh')

In [14]:
_ = tf.keras.utils.plot_model(actor_critic, to_file='logs/model.png')

In [15]:
actor_critic.summary()

Model: "actor_critic"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 7, 7, 6)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 7, 7, 64)     3520        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 7, 7, 64)     256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 7, 7, 64)     0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [16]:
opponent = tf.keras.models.clone_model(actor_critic)

In [17]:
if LOAD_SAVED_MODELS:
    actor_critic.load_weights(ACTOR_CRITIC_PATH)
    opponent.load_weights(ACTOR_CRITIC_PATH)
    logging.info("Loaded saved models")

I0918 14:21:23.020457 4591560128 <ipython-input-17-1004a61459a1>:4] Loaded saved models


In [18]:
state = go_env.get_state()
my_policy = policies.MctPolicy(actor_critic, state, MC_SIMS, TEMP_FUNC)
opponent_policy = policies.MctPolicy(opponent, state, MC_SIMS, TEMP_FUNC)
random_policy = policies.RandomPolicy()

# Demo Trajectories

Symmetries

In [19]:
metrics.plot_symmetries(go_env, actor_critic, 'logs/symmetries.png')

Plot a whole game trajectory

In [20]:
%%time 
traj, _ = data.self_play(go_env, policy=my_policy, max_steps=MAX_STEPS, 
                         get_symmetries=False)

CPU times: user 10.8 s, sys: 306 ms, total: 11.1 s
Wall time: 10.4 s


In [21]:
fig = metrics.gen_traj_fig(go_env, actor_critic, TEMP_FUNC, MAX_STEPS, 
                           MC_SIMS)
fig.savefig('logs/a_trajectory.png')
plt.close()

# Train

In [22]:
actor_critic_opt = tf.keras.optimizers.Adam(LEARNING_RATE)
replay_mem = []

In [24]:
for iteration in range(ITERATIONS):
    # Train
    episode_pbar = tqdm(range(EPISODES_PER_ITERATION), 
                        desc='Iteration {} - Self Play'.format(iteration), 
                        leave=True, position=0)
    for episode in episode_pbar:
        trajectory, num_steps = data.self_play(go_env, policy=my_policy, 
                                               max_steps=MAX_STEPS)
        replay_mem.extend(trajectory)
        tb_metrics['num_steps'].update_state(num_steps)
        
    # Update the models (also shuffles memory)
    random.shuffle(replay_mem)
    np_data = data.replay_mem_to_numpy(replay_mem)
    batched_np_data = [np.array_split(datum, len(replay_mem) // BATCH_SIZE) 
                       for datum in np_data]
    batched_mem = list(zip(*batched_np_data))
    models.update_win_prediction(actor_critic, batched_mem, actor_critic_opt, 
                                 iteration, tb_metrics)
    
    # Evaluate against previous model
    if (iteration+1) % ITERATIONS_PER_EVAL == 0:
        rand_win_rate = metrics.evaluate(go_env, my_policy, random_policy,
                                                 max_steps=MAX_STEPS, 
                                                 num_games=8)
        opp_win_rate = metrics.evaluate(go_env, my_policy, opponent_policy, 
                                    max_steps=MAX_STEPS, 
                                    num_games=NUM_EVAL_GAMES)
        
        stats = "Opp. - {:.1f}%, Rand. - {:.1f}%".format(100*rand_win_rate, 
                                                         100*opp_win_rate)
        if win_rate > 0.6:
            actor_critic.save_weights(ACTOR_CRITIC_PATH)
            opponent.load_weights(ACTOR_CRITIC_PATH)
            
            logging.info("{} Accepted new model".format(stats))
        else:
            logging.info("{} Rejected new model".format(stats))
    
    # Log results and resets the metrics
    
    metrics.log_to_tensorboard(summary_writer, tb_metrics, iteration, go_env, 
                               actor_critic, TEMP_FUNC, 'logs/a_trajectory.png')
    # Reset memory
    replay_mem.clear()

Iteration 0 - Self Play:   6%|▋         | 2/32 [00:18<04:25,  8.84s/it]

ValueError: probabilities do not sum to 1

# Evaluate

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)
state = go_env.reset()
opponent_policy = policies.MctPolicy(network, state, MC_SIMS, TEMP_FUNC)
data.play_against(opponent_policy, go_env)